In [15]:
! pip install streamlit -q


In [16]:
%%writefile app.py
import streamlit as st
import streamlit as st
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf
import matplotlib.pyplot as plt

# Function to predict the next day's closing price
def predict_next_day_closing_price(model, scaler, historical_data):
    last_60_days = historical_data[-60:].values
    last_60_days_scaled = scaler.transform(last_60_days)
    x_test = np.array([last_60_days_scaled])
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    pred_price = model.predict(x_test)
    pred_price = scaler.inverse_transform(pred_price)

    return pred_price[0][0]

# Download historical data for AAPL
start_date = '2001-01-01'
end_date = '2024-02-29'
stock_ticker = 'AAPL'
df = yf.download(stock_ticker, start=start_date, end=end_date)

# Data preprocessing
data = df.filter(['Close'])
dataset = data.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

# Train the LSTM model
training_data_len = int(len(dataset) * 0.8)
train_data = scaled_data[0:training_data_len, :]

x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, batch_size=1, epochs=1)

# Use the trained model to predict the next day's closing price
next_day_pred = predict_next_day_closing_price(model, scaler, data)

# Streamlit App
st.title('Stock Prediction App')
st.markdown("![Alt Text](https://c.tenor.com/jw92b2HUuTAAAAAC/stonks-stocks.gif)")

st.subheader(f'{stock_ticker} Data From {start_date} To {end_date}')
st.write(df)

st.subheader(f'{stock_ticker} Statistical Data')
st.write(df.describe())

st.subheader('Closing Price v/s Time Chart With 100 SMA And 200 SMA')
st.write("A 100-day and 200-day Moving Average (MA) is the average of closing prices of the previous 100 days and 200 days respectively")
st.write("As Per Market Experts -> Buy signal appear when SMA-100 line cut SMA-200 line in its way upward")

ma100 = df.Close.rolling(100).mean()
ma200 = df.Close.rolling(200).mean()
fig = plt.figure(figsize=(12, 6))
plt.plot(df.Close, 'b', label='Closing Price')
plt.plot(ma100, 'r', label='100 SMA')
plt.plot(ma200, 'g', label='200 SMA')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
st.pyplot(fig)

st.subheader('Predict Next Day Closing Price')
st.write(f"Predicted Closing Price for the next day: {next_day_pred}")

# Visualize the predictions
st.subheader('Model Predictions')
plt.figure(figsize=(16, 8))
plt.title('Model Predictions')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price', fontsize=18)
plt.plot(data[-training_data_len:], label='Historical Data')
plt.axvline(x=df.index[-1], color='r', linestyle='--', label='End of Historical Data')
plt.scatter(df.index[-1], next_day_pred, color='g', label='Predicted Next Day Closing Price')
plt.legend()
st.pyplot(plt)

st.write("""
This Is Just For Educational Purpose And No Way A Financial Advice.
Made With ❤️ By [Your Name]
""")


Overwriting app.py


In [ ]:
!pip install pyngrok


In [17]:
! streamlit run app.py & npx localtunnel --port 8501





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.137.15:8501

your url is: https://puny-parents-sleep.loca.lt
2024-10-23 09:19:16.940988: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-23 09:19:16.976778: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-23 09:19:16.987977: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-23 09:19:19.348936: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[****************